In [1]:
import joblib
from collections import Counter
from helper.helper import format_number_and_round
from preprocessing.scarpping_component import extract_component_by_images
import os
import pandas as pd
import dlib
import cv2
from feature_extraction.poc import POC
from feature_extraction.vektor import Vektor
from feature_extraction.quadran import Quadran
from preprocessing.input_to_image import get_frames_by_input_video
import shutil

In [2]:
# Fungsi untuk sorting natural
import re

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]

In [3]:
# Get the result from the predictions
def get_calculate_from_predict(list_decoded_predictions):
    prediction_counts = Counter(list_decoded_predictions)
    total_predictions = len(list_decoded_predictions)
    result_prediction = None
    most_common_count = 0
    list_predictions = []

    for category, count in prediction_counts.items():
        percentage = (count / total_predictions) * 100
        list_predictions.append({
            "name": category,
            "count": count,
            "percentage": format_number_and_round(percentage)
        })
        if count > most_common_count:
            most_common_count = count
            result_prediction = category

    # Sort the predictions by percentage in descending order
    list_predictions.sort(key=lambda x: x["percentage"], reverse=True)

    return result_prediction, list_predictions

In [4]:
# Predict and print results
def predict_and_print_results(df, model, label_encoder, feature_columns):
    df = df.drop(columns=feature_columns)
    predictions = model.predict(df.values)
    decoded_predictions = label_encoder.inverse_transform(predictions)
    result_prediction, list_predictions = get_calculate_from_predict(decoded_predictions)
    
    for pred in list_predictions:
        print(f"Kategori: {pred['name']}, Jumlah: {pred['count']}, Persentase: {pred['percentage']:.2f}%")
    
    print(f"\nHasil Prediksi: {result_prediction}")

In [5]:
# Load the models and label encoders
model_hidung = joblib.load('output-baru-dengan-hidung/data/manhattan/knn_model_n_neighbors_3_metric_manhattan.joblib')
label_encoder_hidung = joblib.load('output-baru-dengan-hidung/data/manhattan/label_encoder_n_neighbors_3_metric_manhattan.joblib')

# Model saved to output-baru-dengan-hidung/data/euclidean/knn_model_n_neighbors_3_metric_euclidean.joblib
# Label encoder saved to output-baru-dengan-hidung/data/euclidean/label_encoder_n_neighbors_3_metric_euclidean.joblib

model_tanpa_hidung = joblib.load('output-baru-tanpa-hidung/data/manhattan/knn_model_n_neighbors_3_metric_manhattan.joblib')
label_encoder_tanpa_hidung = joblib.load('output-baru-tanpa-hidung/data/manhattan/label_encoder_n_neighbors_3_metric_manhattan.joblib')

components_setup = {
    'mulut': {
        'object_name': 'mouth',
        'object_rectangle': {"x_right": 54, "x_left": 48, "y_highest": 52, "y_lowest": 57},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 5},
        'object_dimension': {'width': 140, 'height': 40}
    },
    'mata_kiri': {
        'object_name': 'eye_left',
        'object_rectangle': {"x_right": 39, "x_left": 36, "y_highest": 38, "y_lowest": 41},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
        'object_dimension': {'width': 90, 'height': 55}
    },
    'mata_kanan': {
        'object_name': 'eye_right',
        'object_rectangle': {"x_right": 45, "x_left": 42, "y_highest": 43, "y_lowest": 47},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
        'object_dimension': {'width': 90, 'height': 55}
    },
    'alis_kiri': {
        'object_name': 'eyebrow_left',
        'object_rectangle': {"x_right": 21, "x_left": 17, "y_highest": 18, "y_lowest": 21},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15},
        'object_dimension': {'width': 110, 'height': 40}
    },
    'alis_kanan': {
        'object_name': 'eyebrow_right',
        'object_rectangle': {"x_right": 26, "x_left": 22, "y_highest": 25, "y_lowest": 22},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15},
        'object_dimension': {'width': 110, 'height': 40}
    },
    'hidung_kanan': {
        'object_name': 'nose_right',
        'object_rectangle': {"x_right": 31, "x_left": 40, "y_highest": 40, "y_lowest": 48},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
        'object_dimension': {'width': 70, 'height': 40}
    },
    'hidung_kiri': {
        'object_name': 'nose_left',
        'object_rectangle': {"x_right": 47, "x_left": 35, "y_highest": 47, "y_lowest": 54},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
        'object_dimension': {'width': 70, 'height': 40}
    }
}

# Convert Video to Image
# Path utama
# Definisikan direktori video dan folder output
# video_directory = 'dataset/casme_custom/Disgust/19_EP16_02.avi'

base_directory = 'dataset/casme_custom'
base_output_folder = 'output_consume_video'

# Definisikan direktori video dan folder output
# dimana didalam file casme_custom terdapat folder Disgust, Happiness, Sadness, dan Surprise
emotion_folders = ['Disgust', 'Happiness', 'Sadness', 'Surprise']

for emotion_folder in emotion_folders:
    # Ambil semua file yang ada didalam folder emosi    
    for filename in os.listdir(os.path.join(base_directory, emotion_folder)):
        # Ambil path dari video
        video_directory = os.path.join(base_directory, emotion_folder, filename)

        # Bersihkan folder output jika ada dan buat folder baru
        if os.path.exists(base_output_folder):
            shutil.rmtree(base_output_folder)  # Hapus folder output jika sudah ada
        os.makedirs(base_output_folder)  # Buat folder output baru

        # Cek apakah file ada atau tidak
        if video_directory.endswith(".avi"):

            # Apakah file tersebut ada atau tidak
            if not os.path.exists(video_directory):
                print("Error: File tidak ditemukan.")
                exit
            
            # Buka video menggunakan OpenCV
            vidcap = cv2.VideoCapture(video_directory)

            # Cek apakah video berhasil dibuka
            if not vidcap.isOpened():
                print("Error: Video tidak dapat dibuka.")
                exit()

            # Ambil frame rate dari video
            frame_per_second = vidcap.get(cv2.CAP_PROP_FPS)

            # Variabel untuk menghitung nomor frame
            count = 1

            while True:
                success, image = vidcap.read()  # Baca frame
                if not success:
                    break  # Jika tidak ada frame yang dibaca, keluar dari loop

                # Simpan gambar ke folder output dengan nama imgX.jpg
                img_name = f'img{count}.jpg'
                output_path = os.path.join(base_output_folder, img_name)
                # print(f"Menyimpan {img_name}...")
                cv2.imwrite(output_path, image)

                # Increment count
                count += 1

            # Tutup video
            vidcap.release()
            print(f"Ekstraksi frame dari {video_directory} dengan Total Gambar {count - 1} berhasil. Frame rate: {frame_per_second} FPS.")
        else:
            print("File tidak berformat .avi.")
            # stop cell execution
            exit()


        # Initialize variables for feature extraction
        blockSize = 5
        frames_data_quadran = []
        frames_data_all_component = []
        frames_data_quadran_column = ['sumX', 'sumY', 'Tetha', 'Magnitude', 'JumlahQuadran']
        quadran_dimensions = ['Q1', 'Q2', 'Q3', 'Q4']
        frames_data = {component_name: [] for component_name in components_setup}
        total_blocks_components = {component_name: 0 for component_name in components_setup}
        data_blocks_first_image = {component_name: None for component_name in components_setup}

        # Load face detector and shape predictor
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

        # Hitung total blok dari masing-masing komponen lalu disetup kedalam total_blocks_components
        for component_name, component_info in components_setup.items():
            total_blocks_components[component_name] = int((component_info['object_dimension']['width'] / blockSize) * (component_info['object_dimension']['height'] / blockSize))

        # Reset variabel setiap kali mulai looping folder baru
        data_blocks_first_image = {component_name: None for component_name in components_setup}
        index = {component_name: 0 for component_name in components_setup}

        # looping semua file yang ada didalam
        for filename in sorted(os.listdir(base_output_folder), key=natural_sort_key):
            if filename.endswith(".jpg") or filename.endswith(".png"): 

                # Read path sesuai dengan foldername_join_basepath dijoin path dengan filename
                image = cv2.imread(os.path.join(base_output_folder, filename))
                # Resize image ke ukuran yang diinginkan
                # image = cv2.resize(image, (600, 500))
                # Cpmvert image ke grayscale
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                # Deteksi shape muka didalam grayscale image
                rects = detector(gray)

                if not index[component_name] == 0:
                    # Buat variabel frames_data_all_component untuk menampung data current frame
                    frame_data_all_component = {'Frame': f"{index[component_name] + 1}({filename.split('.')[0]})"}
                    # Buat variabel frame_data_quadran untuk menampung data current frame
                    frame_data_quadran = {'Frame': f"{index[component_name] + 1}({filename.split('.')[0]})"}

                # Memproses rects untuk setiap bentuk wajah yang terdeteksi
                for rect in rects:
                    # Ambil bentuk wajah dalam bentuk shape sesuai dengan model predictor
                    shape = predictor(gray, rect)
                    # Memproses setiap komponen wajah
                    for component_name, component_info in components_setup.items():
                        # print(f"Processing {component_name} in {filename}...")
                        # Inisialisasi variabel sum_data_by_quadran untuk menyimpan data hasil quadran
                        sum_data_by_quadran = {}
                        # Buat variabel frame_data untuk menampung data current frame
                        frame_data = {'Frame': f"{index[component_name] + 1}({filename.split('.')[0]})"}

                        # Looping untuk setiap atribut dalam frames_data_quadran_column
                        for column in frames_data_quadran_column:
                            # Inisialisasi sub-dictionary untuk setiap atribut dalam frames_data_quadran_column yang defaultnya 0
                            sum_data_by_quadran[column] = {quadrant: 0 for quadrant in quadran_dimensions}

                        # Ambil data blok image dari return fungsi extract_component_by_images
                        data_blocks_image_current = extract_component_by_images(
                            image=image,
                            shape=shape,
                            frameName=filename.split(".")[0],
                            objectName=component_info['object_name'],
                            objectRectangle=component_info['object_rectangle'],
                            pixelShifting=component_info['pixel_shifting'],
                            objectDimension=component_info['object_dimension']
                        )
                        
                        # Ambil frame pertama dari perulangan lalu simpan di variabel dan skip (lanjutkan ke frame berikut)
                        if data_blocks_first_image[component_name] is None:
                            # --- Setup bagian 4qmv Dataset ---
                            # Inisialisasi data untuk setiap blok dan setiap kuadran dengan nilai sesuai sum_data_by_quadran
                            # for quadrant in quadran_dimensions:
                            #     for feature in frames_data_quadran_column:
                            #         # Buat nama kolom dengan menggunakan template yang diberikan
                            #         column_name = f"{component_name}_{feature}_{quadrant}"
                            #         # Set value sum_data_by_quadran[feature][quadrant] ke frame_data_quadran sesuai column_name nya
                            #         frame_data_quadran[column_name] = sum_data_by_quadran[feature][quadrant]

                            # --- Setup bagian Nilai Fitur Dataset ---
                            # Inisialisasi data untuk setiap blok
                            # for i in range(total_blocks_components[component_name]):
                                # Tambahkan data ke frame_data sesuai dengan indexnya
                                # frame_data[f'X{i+1}'] = 0
                                # frame_data[f'Y{i+1}'] = 0
                                # frame_data[f'Tetha{i+1}'] = 0
                                # frame_data[f'Magnitude{i+1}'] = 0
                                # # Tambahkan data ke frame_data_all_component sesuai dengan indexnya
                                # frame_data_all_component[f'{component_name}-X{i+1}'] = 0
                                # frame_data_all_component[f'{component_name}-Y{i+1}'] = 0
                                # frame_data_all_component[f'{component_name}-Tetha{i+1}'] = 0
                                # frame_data_all_component[f'{component_name}-Magnitude{i+1}'] = 0

                            # Append data frame ke list frames_data sesuai dengan component_name
                            frames_data[component_name].append(frame_data)
                            # Tambahkan kolom "Folder Path" dengan nilai folder saat ini
                            frame_data['Folder Path'] = 'data_test'
                            # Tambahkan kolom "Label" dengan nilai label saat ini
                            frame_data['Label'] = 'data_test'
                            # Set value data_blocks_first_image[component_name] ke data_blocks_image_current
                            data_blocks_first_image[component_name] = data_blocks_image_current
                            # Skip looping nya ke looping selanjutnya
                            continue

                        # # Tampilkan data block image current ke matplotlib
                        # plt.imshow(np.uint8(data_blocks_image_current), cmap="gray")

                        # Inisiasi class POC
                        initPOC = POC(data_blocks_first_image[component_name], data_blocks_image_current, blockSize) 
                        # Pemanggilan fungsi pocCalc() untuk menghitung nilai POC disetiap gambar
                        valPOC = initPOC.getPOC() 

                        # Pemanggilan class dan method untuk menampilkan quiver / gambar panah
                        initQuiv = Vektor(valPOC, blockSize)
                        quivData = initQuiv.getVektor() 

                        # plt.quiver(quivData[:, 0], quivData[:, 1], quivData[:, 2], quivData[:, 3], scale=1, scale_units='xy', angles='xy', color="r")    

                        # # num = 0
                        # for rect_def in valPOC[2]:
                        #     x, y, width, height = rect_def
                            
                        #     rects = patches.Rectangle((x,y), width,height, edgecolor='r', facecolor='none') 
                        #     plt.gca().add_patch(rects)
                            
                        #     # plt.text(x,y,f'({num})', color="blue") 
                        #     # num += 1

                        # Pemanggilan class untuk mengeluarkan nilai karakteristik vektor
                        # blok ke, x,y,tetha, magnitude, dan quadran ke
                        initQuadran = Quadran(quivData) 
                        quadran = initQuadran.getQuadran()

                        # print(tabulate(quadran, headers=['Blok Ke', 'X', 'Y', 'Tetha', 'Magnitude', 'Quadran Ke']))
                        # plt.axis('on') 
                        # plt.show() 

                        # Update frame_data dengan data quadran
                        for i, quad in enumerate(quadran):
                            # --- Setup bagian Nilai Fitur Dataset ---
                            # Set data kedalam frame_data sesuai column nya
                            frame_data[f'X{i+1}'] = quad[1]
                            frame_data[f'Y{i+1}'] = quad[2]
                            frame_data[f'Tetha{i+1}'] = quad[3]
                            frame_data[f'Magnitude{i+1}'] = quad[4]

                            # Set data kedalam frame_data_all_component sesuai columnnya
                            frame_data_all_component[f'{component_name}-X{i+1}'] = quad[1]
                            frame_data_all_component[f'{component_name}-Y{i+1}'] = quad[2]
                            frame_data_all_component[f'{component_name}-Tetha{i+1}'] = quad[3]
                            frame_data_all_component[f'{component_name}-Magnitude{i+1}'] = quad[4]

                            # --- Setup bagian 4qmv Dataset ---
                            # Cek apakah quad[5] ada didalam array quadran_dimensions
                            if quad[5] in quadran_dimensions:
                                # Tambahkan nilai quad[1] ke sumX pada kuadran yang sesuai
                                sum_data_by_quadran['sumX'][quad[5]] += quad[1]
                                # Tambahkan nilai quad[2] ke sumY pada kuadran yang sesuai
                                sum_data_by_quadran['sumY'][quad[5]] += quad[2]
                                # Tambahkan nilai quad[3] ke Tetha pada kuadran yang sesuai
                                sum_data_by_quadran['Tetha'][quad[5]] += quad[3]
                                # Tambahkan nilai quad[4] ke Magnitude pada kuadran yang sesuai
                                sum_data_by_quadran['Magnitude'][quad[5]] += quad[4]
                                # Tambahkan jumlah quadran sesuai dengan quad[5] ke JumlahQuadran pada kuadran yang sesuai
                                sum_data_by_quadran['JumlahQuadran'][quad[5]] += 1
                        
                        # --- Setup bagian Nilai Fitur Dataset ---
                        # Append data frame ke list
                        frames_data[component_name].append(frame_data)
                        # Tambahkan kolom "Folder Path" dengan nilai folder saat ini
                        frame_data['Folder Path'] = 'data_test'
                        # Tambahkan kolom "Label" dengan nilai label saat ini
                        frame_data['Label'] = 'data_test'

                        # --- Setup bagian 4qmv Dataset ---
                        # Inisialisasi data untuk setiap blok dan setiap kuadran dengan nilai sesuai sum_data_by_quadran
                        for quadran in quadran_dimensions:
                            for feature in frames_data_quadran_column:
                                # Buat nama kolom dengan menggunakan template yang diberikan
                                column_name = f"{component_name}_{feature}_{quadran}"
                                # Set value sum_data_by_quadran[feature][quadran] ke frame_data_quadran sesuai column_name nya
                                frame_data_quadran[column_name] = sum_data_by_quadran[feature][quadran]

                # --- Setup bagian 4qmv Dataset ---
                if not index[component_name] == 0:
                    # Append data frame ke list frames_data_quadran untuk 4qmv
                    frames_data_quadran.append(frame_data_quadran)
                    # print("Frame Quadran", frame_data_quadran)
                    # Tambahkan kolom "Folder Path" dengan nilai folder saat ini
                    frame_data_quadran['Folder Path'] = 'data_test'
                    # Tambahkan kolom "Label" dengan nilai label saat ini
                    frame_data_quadran['Label'] = 'data_test'

                if not index[component_name] == 0:
                    # --- Setup bagian frames data all component Dataset ---
                    # Append data frame ke list frames_data_quadran untuk 4qmv
                    frames_data_all_component.append(frame_data_all_component)
                    # print("Frame Quadran", frame_data_quadran)
                    # Tambahkan kolom "Folder Path" dengan nilai folder saat ini
                    frame_data_all_component['Folder Path'] = 'data_test'
                    # Tambahkan kolom "Label" dengan nilai label saat ini
                    frame_data_all_component['Label'] = 'data_test'

                # Update index per component_name
                index[component_name] += 1

        # Consume Data Ke model joblib
        df_fitur_all = pd.DataFrame(frames_data_all_component)
        except_feature_columns = ['Frame', 'Folder Path', 'Label']

        # Predict and print results with nose
        print("Predictions with Nose:")
        predict_and_print_results(df_fitur_all.copy(), model_hidung, label_encoder_hidung, except_feature_columns)

        # print garis pemisah antar hasil prediksi
        print("=" * 50)

        # Predict and print results without nose (remove nose features)
        nose_features = [col for col in df_fitur_all.columns if 'hidung' in col]
        df_fitur_tanpa_hidung = df_fitur_all.drop(columns=nose_features)

        print("Predictions without Nose:")
        predict_and_print_results(df_fitur_tanpa_hidung.copy(), model_tanpa_hidung, label_encoder_tanpa_hidung, except_feature_columns)


        # hasil pemisah antar file
        print("\n" + "#" * 50 + "\n")

        # hapus dataframes
        del df_fitur_all
        del df_fitur_tanpa_hidung
        del frames_data_all_component
        del frames_data_quadran
        del frames_data
        del data_blocks_first_image
        del index

Ekstraksi frame dari dataset/casme_custom/Disgust/24_EP02_02f.avi dengan Total Gambar 488 berhasil. Frame rate: 200.0 FPS.
Predictions with Nose:
Kategori: Disgust, Jumlah: 487, Persentase: 100.00%

Hasil Prediksi: Disgust
Predictions without Nose:
Kategori: Disgust, Jumlah: 487, Persentase: 100.00%

Hasil Prediksi: Disgust

##################################################

Ekstraksi frame dari dataset/casme_custom/Disgust/19_EP16_02.avi dengan Total Gambar 266 berhasil. Frame rate: 200.0 FPS.
Predictions with Nose:
Kategori: Disgust, Jumlah: 265, Persentase: 100.00%

Hasil Prediksi: Disgust
Predictions without Nose:
Kategori: Disgust, Jumlah: 265, Persentase: 100.00%

Hasil Prediksi: Disgust

##################################################

Ekstraksi frame dari dataset/casme_custom/Disgust/03_EP19_08.avi dengan Total Gambar 291 berhasil. Frame rate: 200.0 FPS.
Predictions with Nose:
Kategori: Disgust, Jumlah: 290, Persentase: 100.00%

Hasil Prediksi: Disgust
Predictions without N

KeyError: "['Frame', 'Folder Path', 'Label'] not found in axis"